In [1]:
!pip install pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.6 MB/s eta 0:00:00


In [29]:
import pymysql
from urllib.parse import quote
from urllib.request import Request,urlopen
import json
import re
import pandas as pd

from bs4 import BeautifulSoup
import requests
import os
from datetime import datetime, timedelta
from dateutil import tz

In [30]:
def str_filter(text):
    html_spch = ['&quot;','&amp;','&lt;','&gt;','&apos;',
             '&nbsp;','&iexcl;','&cent;','&pound;',
             '&curren;','&yen;','&brvbar;','&sect;',
             '&uml;','&copy;','&ordf;','&laquo;','&not;',
             '&shy;','&reg;','&macr;','&deg;','&plusmn;',
             '&sup2;','&sup3;','&acute;','&micro;','&para;',
             '&middot;','&cedil;','&sup1;','&ordm;','&raquo;',
             '&frac14;','&frac12;','&frac34;','&iquest;']
    html_tag = ['<b>','\n','</b>','<b/>','<a>','</a>','<a/>',
            '<br>','</br>','<br/>','<p>','</p>','<p/>',
            '<strong>','</strong>','<strong/>']
    html_spch_tag = html_spch + html_tag
    or_exp = '|'.join(html_spch_tag)
    text = re.sub(or_exp," ",text)
    text1= re.sub(r'[^\w\s]',' ',text)
    text2= re.sub(r"^\s+|\ㄴ+$","",text1) # 양측 공백 제거
    return text2

In [31]:
def extract_naverBlog(url):
  e_img_urls=""
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  ifra = soup.find('iframe', id='mainFrame')
  post_url = 'https://blog.naver.com' + ifra['src']

  res = requests.get(post_url)
  soup2 = BeautifulSoup(res.text, 'html.parser')

  # 날짜 추출
  post_date= soup2.find_all('span',class_='se_publishDate pcol2')[0].get_text()

  # 제목 추출
  titles = soup2.find_all('div', {'class': re.compile('^se-module se-module-text se-title-tex.*')})
  post_title = titles[0].text
  post_title = post_title.replace('\n', '')
  post_title=str_filter(post_title) # 특수 문자 제거

  #본문 추출
  contents = ''
  txt_contents = soup2.find_all('div', {'class': re.compile('^se-module se-module-tex.*')})
  for p_span in txt_contents:
    for txt in p_span.find_all('span'):
      contents += txt.get_text() + '\n'

  main_text=str_filter(contents) # 특수 문자 제거
  main_text=main_text[len(post_title):]

  # 이미지 링크 가져오기
  end_img_urls=[]
  imgs = soup2.find_all('img', class_='se-image-resource')

  for i in [-1]:
    if imgs[i].get('data-lazy-src')==None:
      end_img_urls.append(imgs[i].get('src'))
    else:
      end_img_urls.append(imgs[i].get('data-lazy-src'))

  # php 협찬 사이트
  php_links = soup2.find_all('img', class_='se-inline-image-resource')

  # 네이버 스티커
  Naver_sticker_urls=[]
  Naver_sticker= soup2.find_all('img', class_='se-sticker-image')
  for i in [-1]:
    try:
      Naver_sticker_urls.append(Naver_sticker[i].get('src'))
    except:
      pass

  end_img_urls.extend( Naver_sticker_urls)


  #### 블로그 내 지도 제목 가져오기
  maps=soup2.find_all('strong', {'class': re.compile('se-map-title') })
  map_titles=[]
  for map in maps:
    map_titles.append(map.text.replace('\n',''))

  ## url 로 ad 거르기
  ad_words = ['seoulouba', 'mrblog', 'revu', 'dinnerqueen', 'xn--939au0g4vj8sq']
  ad_score=0
  for ad_word in ad_words:
    for php_link in php_links:
      if ad_word in php_link['src']:
        ad_score=1

  for ad_word in ad_words:
    for img_url in end_img_urls:
      if img_url.find(ad_word)>-1:
        ad_score=1

  return (url, post_title, main_text, end_img_urls, ad_score, post_date, map_titles)

In [32]:
conn = pymysql.connect(host='blogdb.cm2yxwfja9ii.ap-northeast-2.rds.amazonaws.com',
                      user='admin',
                      password='blogdb!2',
                      database='Naver_Blogs',
                      port=3306)
curs=conn.cursor()

In [33]:
query="SELECT blog_ID,url FROM post WHERE content='' ORDER BY blog_ID "

curs.execute(query)

# Fetch all the rows
data = curs.fetchall()
#range(len(data))

In [35]:
len(data)

114863

In [ ]:
# 네이버 api로 얻은 블로그 url 을 이용하여 본문 텍스트, 이미지 업데이트
# 1000 개 하는데 25분
for dat in data:
  blog_id=dat[0]
  blog_url=dat[1]
  blog_url1=blog_url
  print()
  print(blog_id)
  print(blog_url)

  if blog_url.find('blog')>-1:
    blog_url='https://'+blog_url.split('m.')[1]
    try:
      blog_info = extract_naverBlog(blog_url)
    except:
      print('extract_naverBlog 실패')

    title=blog_info[1]
    main_text=blog_info[2]
    i_urls=blog_info[3]
    ad_score=blog_info[4]
    post_date=blog_info[5]

    if post_date.find('시간')>-1:
      p_date=post_date[:post_date.find('시간')]
      p_date=int(p_date)

      utc_zone = tz.gettz('UTC')
      seoul_zone = tz.gettz('Asia/Seoul')
      utc = datetime.now()
      current_time = utc.replace(tzinfo=utc_zone).astimezone(seoul_zone)

      post_date= current_time - timedelta(hours=p_date)
      post_date=post_date.strftime("%Y. %m. %d. %H:%M")

    for url in i_urls:
      update_sql2= f"INSERT INTO blog_img_url (blog_id, img_url) VALUES ({blog_id},'{url}');"
      try:
        curs.execute(update_sql2)
        conn.commit()
      except Exception as e:
        print(f"An error occurred: {e}")
        conn.rollback()  # Rollback changes if an error occurs

    update_sql = f"UPDATE post SET post_date=DATE_FORMAT(STR_TO_DATE('{post_date}', '%Y. %m. %d. %H:%i'), '%Y-%m-%d'), title='{title}', content='{main_text}', ad={ad_score} WHERE url='{blog_url1}';"

    try:
      curs.execute(update_sql)
      conn.commit()
    except Exception as e:
      print(f"An error occurred: {e}")
      conn.rollback()  # Rollback changes if an error occurs

    #초기화
    post_date=''
    title=''
    main_text=''
    i_urls=''
    ad_score=0
  else:
    #초기화
    post_date=''
    title=''
    main_text=''
    i_urls=''
    ad_score=0
